In [1]:
import os
import cv2
import mediapipe as mp
import numpy as np
import pickle

In [2]:
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

hands = mp_hands.Hands(
    static_image_mode=False,      # Faster than True
    max_num_hands=1,
    min_detection_confidence=0.3
)

# ASL dataset
DATA_DIR = "../data2"

data = []
labels = []

# Counter for progress logging
global_counter = 0
PRINT_EVERY = 500  

In [3]:
for label in os.listdir(DATA_DIR):
    folder_path = os.path.join(DATA_DIR, label)

    if not os.path.isdir(folder_path):
        continue

    class_count = 0  

    # Loop over each image in that folder
    for img_name in os.listdir(folder_path):
        img_path = os.path.join(folder_path, img_name)
        img = cv2.imread(img_path)

        if img is None:
            continue  

        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img_rgb = cv2.resize(img_rgb, (320, 320))   # you can try 224x224 too

        results = hands.process(img_rgb)

        if results.multi_hand_landmarks:  
            hand_landmarks = results.multi_hand_landmarks[0]

            # Extract 21 (x, y) keypoints
            keypoints = []
            for lm in hand_landmarks.landmark:
                keypoints.extend([lm.x, lm.y])

            data.append(keypoints)
            labels.append(label)
            class_count += 1
            global_counter += 1

            # Print progress every 500th processed image
            if global_counter % PRINT_EVERY == 0:
                print(f"[{global_counter}] Processed image → Class '{label}' (class count: {class_count})")

    print(f"Finished class '{label}' → {class_count} samples extracted", flush=True)

# Save dataset
with open("../asl_keypoints_dataset.pickle", "wb") as f:
    pickle.dump({"data": data, "labels": labels}, f)

print(f"\nDone! Total samples saved: {len(data)} into asl_keypoints_dataset.pickle")

[500] Processed image → Class 'A' (class count: 500)
[1000] Processed image → Class 'A' (class count: 1000)


KeyboardInterrupt: 